In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import torch
torch.cuda.is_available()


True

In [3]:
df = pd.read_csv("../data/cleaned_modv2.csv")

In [4]:
df['user_id'] = df['user_id'].astype('category')
df['time_signature'] = df['time_signature'].astype(int)
df['explicit'] = df['explicit'].astype(int)


In [5]:
df.head(2)

,user_id,age,gender,music,artist_name,featured_artists,genre,plays,duration,music_id,...,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,time_signature,explicit
0,83811,16,F,Bank Account,21 Savage,"Birdy, Zoé",Dark Trap,11,3.67,2fQrGHiQOvpL9UgPvtYy6G,...,8.0,-8.228,0.0,0.3510,0.000007,0.0871,0.376,75.0160,4,1
1,83811,16,F,Little Talks,Of Monsters and Men,"Ninho, Snoop Dogg, Russ, Paramore",Unknown,686,4.44,2ihCaVdNZmnHZWt0fvAM7B,...,1.0,-7.879,1.0,0.0322,0.000000,0.2845,0.413,101.8905,4,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19974 entries, 0 to 19973
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   user_id           19974 non-null  category
 1   age               19974 non-null  int64   
 2   gender            19974 non-null  object  
 3   music             19974 non-null  object  
 4   artist_name       19974 non-null  object  
 5   featured_artists  19974 non-null  object  
 6   genre             19974 non-null  object  
 7   plays             19974 non-null  int64   
 8   duration          19974 non-null  float64 
 9   music_id          19974 non-null  object  
 10  id_artists        19974 non-null  object  
 11  acousticness      19974 non-null  float64 
 12  danceability      19974 non-null  float64 
 13  energy            19974 non-null  float64 
 14  key               19974 non-null  float64 
 15  loudness          19974 non-null  float64 
 16  mode              1997

In [7]:
numerical_features = df.select_dtypes(include=[np.number])
numerical_features

,age,plays,duration,acousticness,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,time_signature,explicit
0,16,11,3.67,0.015100,0.884000,0.346000,8.0,-8.228000,0.0,0.351000,0.000007,0.087100,0.376000,75.016000,4,1
1,16,686,4.44,0.180850,0.577500,0.497500,1.0,-7.879000,1.0,0.032200,0.000000,0.284500,0.413000,101.890500,4,0
2,17,136,2.83,0.363482,0.568667,0.644000,0.0,-7.344833,1.0,0.089400,0.043432,0.273133,0.404183,120.850500,4,0
3,44,230,5.14,0.676000,0.843000,0.419000,1.0,-7.606000,1.0,0.167000,0.000000,0.096600,0.400000,109.283000,4,0
4,44,391,3.20,0.359471,0.518229,0.544046,0.0,-9.894632,1.0,0.078837,0.235527,0.180354,0.380815,120.488479,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19969,18,890,4.03,0.752000,0.528000,0.355000,9.0,-11.712000,1.0,0.046400,0.091100,0.427000,0.677000,123.014000,4,0
19970,18,746,3.26,0.849000,0.529000,0.112000,9.0,-13.413000,0.0,0.033800,0.005650,0.095200,0.211000,79.858000,4,0
19971,44,176,4.10,0.221401,0.579714,0.724286,1.0,-6.716571,1.0,0.058614,0.003260,0.156643,0.567000,113.667714,4,0
19972,65,394,3.93,0.079100,0.506000,0.799000,1.0,-6.580000,1.0,0.380000,0.000000,0.092900,0.461000,120.405000,5,1


In [8]:
categorical_features = df.select_dtypes(include=[object])
categorical_features


,gender,music,artist_name,featured_artists,genre,music_id,id_artists
0,F,Bank Account,21 Savage,"Birdy, Zoé",Dark Trap,2fQrGHiQOvpL9UgPvtYy6G,spotify:artist:1URnnhqYAYcrqrcwql10ft
1,F,Little Talks,Of Monsters and Men,"Ninho, Snoop Dogg, Russ, Paramore",Unknown,2ihCaVdNZmnHZWt0fvAM7B,spotify:artist:4dwdTW1Lfiq0cM8nBAqIIz
2,M,Wherever I Go,OneRepublic,"Keith Urban, DJ Khaled, NIKI, MF DOOM",Unknown,46jLy47W8rkf8rEX04gMKB,spotify:artist:5Pwc4xIPtQLFEnJriah9YJ
3,M,No New Friends,DJ Khaled,"The xx, LIT killah",Pop,5oVlbbiKGdGeZkWCFy0mqk,spotify:artist:0QHgL1lAIqAw0HtD7YldmP
4,M,Dreams,Campsite Dream,none,Country,1SNoSoQ3JZldOhzBY9gw0n,spotify:artist:69VkQLf4DH7GJ68BCDOPKL
...,...,...,...,...,...,...,...
19969,F,Houses Of The Holy,Led Zeppelin,"Dido, Van Halen, Bazzi, Years & Years, Bibi Bl...",Dance/Electronic,4es5wreov9D4Y4fXLGZkuB,spotify:artist:36QJpDe2go2KgaRleHCDTp
19970,F,Halfway Gone,Lifehouse,none,Rock,7JtQLJcOyVJKUPdUC4lggE,spotify:artist:5PokPZn11xzZXyXSfnvIM3
19971,M,Take You Down,Chris Brown,Florida Georgia Line,Rock,3wu9ADop1FXdhToPCxwBL8,spotify:artist:7bXgB6jMjp9ATFy66eO08Z
19972,F,We Fly High,Jim Jones,none,Christian/Gospel,5rXkgeY6rpcHcsolCpXZHR,spotify:artist:6AMa1VFQ7qCi61tCRtVWXe


In [9]:
data = df.copy()

In [10]:
# Prepare user features
user_features = ['user_id', 'age', 'gender']
user_features_mapped = pd.get_dummies(data[user_features], columns=['gender'])

# Prepare item features
item_features = ['music_id', 'artist_name', 'genre']
item_features_mapped = pd.get_dummies(data[item_features], columns=['genre'])

# Prepare audio features
audio_features = ['acousticness', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                  'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'explicit']
audio_features_scaled = (data[audio_features] - data[audio_features].mean()) / data[audio_features].std()

# Combine user, item, and audio features
features = pd.concat([user_features_mapped, item_features_mapped, audio_features_scaled], axis=1)

# Prepare target variable
target = data['plays']

In [11]:
# Split data into train and test sets
train_data = features.sample(frac=0.8, random_state=42)
test_data = features.drop(train_data.index)

train_target = target[train_data.index]
test_target = target[test_data.index]

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19974 entries, 0 to 19973
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   user_id           19974 non-null  category
 1   age               19974 non-null  int64   
 2   gender            19974 non-null  object  
 3   music             19974 non-null  object  
 4   artist_name       19974 non-null  object  
 5   featured_artists  19974 non-null  object  
 6   genre             19974 non-null  object  
 7   plays             19974 non-null  int64   
 8   duration          19974 non-null  float64 
 9   music_id          19974 non-null  object  
 10  id_artists        19974 non-null  object  
 11  acousticness      19974 non-null  float64 
 12  danceability      19974 non-null  float64 
 13  energy            19974 non-null  float64 
 14  key               19974 non-null  float64 
 15  loudness          19974 non-null  float64 
 16  mode              1997

In [13]:
# convert from object to category
objects = data.select_dtypes(include=[object])
for column in objects:
    data[column] = data[column].astype('category')

    

In [14]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming your DataFrame is named 'data'

# Perform label encoding for 'user_id' and 'music_id' columns
user_id_encoder = LabelEncoder()
music_id_encoder = LabelEncoder()
data['user_id_encoded'] = user_id_encoder.fit_transform(data['user_id'])
data['music_id_encoded'] = music_id_encoder.fit_transform(data['music_id'])

# Perform label encoding for 'gender' column
gender_encoder = LabelEncoder()
data['gender_encoded'] = gender_encoder.fit_transform(data['gender'])

# Perform TF-IDF encoding for 'artist_name' and 'genre' columns
tfidf_vectorizer = TfidfVectorizer()
artist_tfidf = tfidf_vectorizer.fit_transform(data['artist_name'])
genre_tfidf = tfidf_vectorizer.fit_transform(data['genre'])

# Convert TF-IDF matrices to DataFrames
artist_tfidf_df = pd.DataFrame(artist_tfidf.toarray(), columns=[f'artist_tfidf_{i}' for i in range(artist_tfidf.shape[1])])
genre_tfidf_df = pd.DataFrame(genre_tfidf.toarray(), columns=[f'genre_tfidf_{i}' for i in range(genre_tfidf.shape[1])])

# Concatenate the encoded DataFrames with the original DataFrame
data_encoded = pd.concat([data[['user_id_encoded', 'music_id_encoded', 'age', 'gender_encoded', 'duration', 'acousticness', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'explicit', 'plays']], artist_tfidf_df, genre_tfidf_df], axis=1)

# Split the data into features and target
features = data_encoded.drop(columns=['plays'])
target = data_encoded['plays']

# Split the data into train and test sets
train_data = features.sample(frac=0.8, random_state=42)
test_data = features.drop(train_data.index)

train_target = target[train_data.index]
test_target = target[test_data.index]

# Convert data to PyTorch tensors
train_data = torch.tensor(train_data.values, dtype=torch.float)
test_data = torch.tensor(test_data.values, dtype=torch.float)
train_target = torch.tensor(train_target.values, dtype=torch.float).unsqueeze(1)
test_target = torch.tensor(test_target.values, dtype=torch.float).unsqueeze(1)

In [15]:
# verify the data 
print(f"Train data shape: {train_data.shape}\nTest data shape: {test_data.shape}\nTrain target shape: {train_target.shape}\nTest target shape: {test_target.shape}")

Train data shape: torch.Size([15979, 6101])
Test data shape: torch.Size([3995, 6101])
Train target shape: torch.Size([15979, 1])
Test target shape: torch.Size([3995, 1])


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the ListNet loss
class ListNetLoss(nn.Module):
    def __init__(self, k=10):
        super(ListNetLoss, self).__init__()
        self.k = k

    def forward(self, y_pred, y_true):
        y_pred = F.softmax(y_pred, dim=1)
        y_true = F.softmax(y_true, dim=1)
        return -torch.sum(y_true * torch.log(y_pred), dim=1).mean()

# Define the hybrid recommender model
class HybridRecommender(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, num_features, num_layers, hidden_dims, dropout_prob):
        super(HybridRecommender, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        layers = []
        input_dim = embedding_dim * 2 + num_features
        for i in range(num_layers):
            layers.append(nn.Linear(input_dim, hidden_dims[i]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=dropout_prob))
            input_dim = hidden_dims[i]
        layers.append(nn.Linear(input_dim, 1))

        self.fc_layers = nn.Sequential(*layers)

    def forward(self, user_ids, item_ids, features):
        user_embedded = self.user_embedding(user_ids)
        item_embedded = self.item_embedding(item_ids)
        concat_features = torch.cat((user_embedded, item_embedded, features), dim=1)
        output = self.fc_layers(concat_features)
        return output.squeeze()

# Assuming your DataFrame is named 'data'

# Perform label encoding for 'user_id' and 'music_id' columns
user_id_encoder = LabelEncoder()
music_id_encoder = LabelEncoder()
data['user_id_encoded'] = user_id_encoder.fit_transform(data['user_id'])
data['music_id_encoded'] = music_id_encoder.fit_transform(data['music_id'])

# Perform label encoding for 'gender' column
gender_encoder = LabelEncoder()
data['gender_encoded'] = gender_encoder.fit_transform(data['gender'])

# Perform TF-IDF encoding for 'artist_name' and 'genre' columns
tfidf_vectorizer = TfidfVectorizer()
artist_tfidf = tfidf_vectorizer.fit_transform(data['artist_name'])
genre_tfidf = tfidf_vectorizer.fit_transform(data['genre'])

# Convert TF-IDF matrices to DataFrames
artist_tfidf_df = pd.DataFrame(artist_tfidf.toarray(), columns=[f'artist_tfidf_{i}' for i in range(artist_tfidf.shape[1])])
genre_tfidf_df = pd.DataFrame(genre_tfidf.toarray(), columns=[f'genre_tfidf_{i}' for i in range(genre_tfidf.shape[1])])

# Concatenate the encoded DataFrames with the original DataFrame
data_encoded = pd.concat([data[['user_id_encoded', 'music_id_encoded', 'age', 'gender_encoded', 'duration', 'acousticness', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'explicit', 'plays']], artist_tfidf_df, genre_tfidf_df], axis=1)

# Split the data into features and target
features = data_encoded.drop(columns=['plays'])
target = data_encoded['plays']

# Split the data into train and test sets
train_data = features.sample(frac=0.8, random_state=42)
test_data = features.drop(train_data.index)

train_target = target[train_data.index]
test_target = target[test_data.index]

# Convert data to PyTorch tensors
train_data_tensor = torch.tensor(train_data.values, dtype=torch.float)
test_data_tensor = torch.tensor(test_data.values, dtype=torch.float)
train_target_tensor = torch.tensor(train_target.values, dtype=torch.float).unsqueeze(1)
test_target_tensor = torch.tensor(test_target.values, dtype=torch.float).unsqueeze(1)

# Extract user IDs, item IDs, and feature tensors
train_user_ids = train_data_tensor[:, 0].long()
train_item_ids = train_data_tensor[:, 1].long()
train_features = train_data_tensor[:, 2:]

test_user_ids = test_data_tensor[:, 0].long()
test_item_ids = test_data_tensor[:, 1].long()
test_features = test_data_tensor[:, 2:]

# Initialize the model
num_users = len(user_id_encoder.classes_)
num_items = len(music_id_encoder.classes_)
embedding_dim = 128
num_layers = 3
hidden_dims = [256, 128, 64]
num_features = train_features.shape[1]
dropout_prob = 0.2  # Ensure dropout_prob is defined

model = HybridRecommender(num_users, num_items, embedding_dim, num_features, num_layers, hidden_dims, dropout_prob)

# Train the model with ListNet loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

criterion = ListNetLoss(k=10)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# Train the model with regularization and dropout
num_epochs = 30
batch_size = 128
dropout_prob = 0.2
for epoch in range(num_epochs):
    for i in range(0, len(train_data), batch_size):
        batch_user_ids = train_user_ids[i:i+batch_size].to(device)
        batch_item_ids = train_item_ids[i:i+batch_size].to(device)
        batch_features = train_features[i:i+batch_size].to(device)
        batch_target = train_target_tensor[i:i+batch_size].to(device)

        optimizer.zero_grad()
        predictions = model(batch_user_ids, batch_item_ids, batch_features)
        loss = criterion(predictions.unsqueeze(1), batch_target)
        # Add L2 regularization to the loss
        l2_reg = torch.tensor(0.).to(device)
        for param in model.parameters():
            l2_reg += torch.norm(param)
        loss += 1e-4 * l2_reg
        loss.backward()
        optimizer.step()
    scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model with NDCG@k
def ndcg_at_k(y_true, y_pred, k):
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)

    indices = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[indices]

    gains = np.log2(y_true_sorted + 1)
    discounts = np.log2(np.arange(len(y_true_sorted)) + 2)
    dcg = np.sum(gains[:k] / discounts[:k])

    ideal_gains = np.log2(np.sort(y_true)[::-1] + 1)
    ideal_dcg = np.sum(ideal_gains[:k] / discounts[:k])

    return dcg / ideal_dcg

with torch.no_grad():
    # Move input tensors to the same device as the model
    predictions = model(test_user_ids.to(device), test_item_ids.to(device), test_features.to(device))  
    ndcg_score = ndcg_at_k(test_target_tensor.cpu().numpy(), predictions.cpu().numpy(), k=10) # Move target tensor to cpu for ndcg calculation
    print(f"Test NDCG@10: {ndcg_score:.4f}")

Epoch [1/30], Loss: 0.2130
Epoch [2/30], Loss: 0.1937
Epoch [3/30], Loss: 0.1753
Epoch [4/30], Loss: 0.1733
Epoch [5/30], Loss: 0.1715
Epoch [6/30], Loss: 0.1698
Epoch [7/30], Loss: 0.1696
Epoch [8/30], Loss: 0.1694
Epoch [9/30], Loss: 0.1692
Epoch [10/30], Loss: 0.1692
Epoch [11/30], Loss: 0.1692
Epoch [12/30], Loss: 0.1692
Epoch [13/30], Loss: 0.1692
Epoch [14/30], Loss: 0.1692
Epoch [15/30], Loss: 0.1691
Epoch [16/30], Loss: 0.1691
Epoch [17/30], Loss: 0.1691
Epoch [18/30], Loss: 0.1691
Epoch [19/30], Loss: 0.1691
Epoch [20/30], Loss: 0.1691
Epoch [21/30], Loss: 0.1691
Epoch [22/30], Loss: 0.1691
Epoch [23/30], Loss: 0.1691
Epoch [24/30], Loss: 0.1691
Epoch [25/30], Loss: 0.1691
Epoch [26/30], Loss: 0.1691
Epoch [27/30], Loss: 0.1691
Epoch [28/30], Loss: 0.1691
Epoch [29/30], Loss: 0.1691
Epoch [30/30], Loss: 0.1691
Test NDCG@10: 0.3905


In [17]:
# save the model
torch.save(model.state_dict(), 'model.pth')

### Generate recommendations for a user

In [17]:
import torch
import numpy as np

def get_recommendations(model, user_id, data_encoded, user_id_encoder, item_encoder, device, top_k=10):
    """
    Generate top-k recommendations for a given user.

    Args:
        model (nn.Module): Trained recommender model.
        user_id (str): The user ID for whom to generate recommendations.
        data_encoded (pd.DataFrame): The preprocessed and encoded dataset.
        user_id_encoder (LabelEncoder): Fitted LabelEncoder for user IDs.
        item_encoder (LabelEncoder): Fitted LabelEncoder for item IDs.
        device (torch.device): Device where the model is loaded.
        top_k (int): Number of top recommendations to return.

    Returns:
        List[str]: List of recommended music IDs.
    """
    model.eval()  # Set model to evaluation mode

    # Encode the user ID
    try:
        user_id_encoded = user_id_encoder.transform([user_id])[0]
    except ValueError:
        raise ValueError(f"User ID {user_id} not found in encoder.")

    # Extract user features
    user_data = data_encoded[data_encoded['user_id_encoded'] == user_id_encoded].drop(columns=['user_id_encoded', 'plays']).iloc[0]
    user_features = torch.tensor(user_data.values, dtype=torch.float).unsqueeze(0).to(device)

    # Prepare all item IDs
    item_ids = torch.arange(len(item_encoder.classes_)).to(device)

    # Create user and item tensors
    user_ids = torch.tensor([user_id_encoded] * len(item_ids), dtype=torch.long).to(device)
    item_ids = item_ids.long()

    # Disable gradient computation
    with torch.no_grad():
        # Forward pass to get scores
        scores = model(user_ids, item_ids, user_features.repeat(len(item_ids), 1))

    # Get top-k scores and corresponding item IDs
    top_scores, top_indices = torch.topk(scores, top_k)
    top_item_ids_encoded = top_indices.cpu().numpy()

    # Convert encoded item IDs back to original IDs
    top_item_ids = item_encoder.inverse_transform(top_item_ids_encoded)

    return top_item_ids.tolist()

In [18]:
data['user_id'].head()

0    83811
1    83811
2    13397
3    70645
4    70645
Name: user_id, dtype: category
Categories (9741, int64): [4, 16, 35, 74, ..., 99980, 99996, 99997, 99998]

In [19]:
# Assuming the following variables are already defined and loaded:
# model, user_id_encoder, item_encoder, data_encoded, device

user_id = "83811"
top_k = 10

try:
    recommended_items = get_recommendations(model, user_id, data_encoded, user_id_encoder, item_encoder, device, top_k)
    print(f"Top {top_k} recommendations for user {user_id}:")
    for item in recommended_items:
        print(item)
except ValueError as ve:
    print(ve)

NameError: name 'item_encoder' is not defined

In [20]:
test_user_ids_sample = test_user_ids[:10]  # Adjust the number of samples as needed
test_item_ids_sample = test_item_ids[:10]
test_features_sample = test_features[:10]

with torch.no_grad():
    sample_predictions = model(test_user_ids_sample.to(device), test_item_ids_sample.to(device), test_features_sample.to(device))
    print("Sample Predictions:", sample_predictions.cpu().numpy())


Sample Predictions: [1.317741e-13 1.317741e-13 1.317741e-13 1.317741e-13 1.317741e-13
 1.317741e-13 1.317741e-13 1.317741e-13 1.317741e-13 1.317741e-13]


In [21]:
def make_inference(model, user_id, item_id, features, user_encoder, item_encoder, device):
    model.eval()  # Set the model to evaluation mode
    user_id_encoded = torch.tensor(user_encoder.transform([user_id]), dtype=torch.long).to(device)
    item_id_encoded = torch.tensor(item_encoder.transform([item_id]), dtype=torch.long).to(device)
    features_tensor = torch.tensor(features, dtype=torch.float).unsqueeze(0).to(device)

    with torch.no_grad():
        prediction = model(user_id_encoded, item_id_encoded, features_tensor)
    return prediction.cpu().numpy()


In [22]:
data['music_id_encoded']

0         3936
1         4020
2         6090
3         8577
4         2130
         ...  
19969     6901
19970    10826
19971     5836
19972     8646
19973     9213
Name: music_id_encoded, Length: 19974, dtype: int64

In [23]:
data['music_id']


0        2fQrGHiQOvpL9UgPvtYy6G
1        2ihCaVdNZmnHZWt0fvAM7B
2        46jLy47W8rkf8rEX04gMKB
3        5oVlbbiKGdGeZkWCFy0mqk
4        1SNoSoQ3JZldOhzBY9gw0n
                  ...          
19969    4es5wreov9D4Y4fXLGZkuB
19970    7JtQLJcOyVJKUPdUC4lggE
19971    3wu9ADop1FXdhToPCxwBL8
19972    5rXkgeY6rpcHcsolCpXZHR
19973    6Ep6BzIOB9tz3P4sWqiiAB
Name: music_id, Length: 19974, dtype: category
Categories (11528, object): ['0010mZpCCwlPwoBiBsjoac', '0088Tt3QK3fMYodhkIEoUh', '00AivYmu1UVmxM91uhR9lM', '00BuKLSAFkaEkaVAgIMbeA', ..., '7zqB9qTuzWHwhddWf5FwWQ', '7zuwaenG5AF0vG7o7kMduX', '7zxRMhXxJMQCeDDg0rKAVo', '7zyja1IM5jwetBYGZJhxXP']

In [24]:
def get_recommendations(user_id, top_k=10):
    user_id_encoded = user_id_encoder.transform([user_id])[0]
    user_data = data_encoded[data_encoded['user_id_encoded'] == user_id_encoded].drop(columns=['user_id_encoded', 'plays']).iloc[0]
    user_tensor = torch.tensor(user_data.values, dtype=torch.float).unsqueeze(0)
    
    item_ids = torch.tensor(range(num_items), dtype=torch.long)
    item_embeddings = model.item_embedding(item_ids)
    user_embedding = model.user_embedding(torch.tensor([user_id_encoded], dtype=torch.long))
    user_embedding_repeated = user_embedding.repeat(num_items, 1)
    
    scores = model.fc_layers(torch.cat((user_embedding_repeated, item_embeddings, user_tensor.repeat(num_items, 1)), dim=1))
    
    top_items = item_ids[scores.argsort(descending=True)][:top_k]
    return top_items.tolist()